In [1]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

# Udahub Application

## Core Database

**Init DB**

In [2]:
udahub_db = "data/core/udahub.db"

In [3]:
reset_db(udahub_db)

✅ Removed existing data/core/udahub.db
2025-11-30 17:40:14,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-30 17:40:14,896 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/core/udahub.db with fresh schema


In [4]:
engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(bind=engine)

**Account**

In [5]:
account_id = "cultpass"
account_name = "CultPass Card"

In [6]:
with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

## Integrations

**Knowledge Base**

In [7]:
cultpass_articles = []

with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

In [8]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

**Use LLM to Generate Sample Data**

In [9]:
import os
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel, Field
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_base_url = "https://openai.vocareum.com/v1"

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    base_url="https://openai.vocareum.com/v1",
    api_key=OPENAI_API_KEY,
)

In [10]:
# TODO: Create additional 10 articles
class CultpassArticlesSchema(BaseModel):
    title: str = Field(description="Title of the article.")
    content: str = Field(description="Content of the article.")
    tags: str = Field(description="Tags of the article")

class CultpassArticlesListSchema(BaseModel):
    samples: List[CultpassArticlesSchema] = Field(description="List of CultpassArticles")

sample_articles_template = """
            You are an helpful expert creating sample of python sample article data based on given examples.  Do not provide any additional description or explanation.
            There is a mock-up project, it has the following description:

            You’ve joined a fast-growing AI startup building the next frontier in customer support automation. Your team is responsible for building UDA-Hub, 
            a Universal Decision Agent designed to plug into existing customer support systems (Zendesk, Intercom, Freshdesk, internal CRMs) and intelligently 
            resolve tickets. But this isn’t just another FAQ bot.

            The goal? Build an agentic system that reads, reasons, routes, and resolves, acting as the operational brain behind support teams.

            You’ll need to design an agent system that can:
                - Understand customer tickets across channels
                - Decide which agent or tool should handle each case
                - Retrieve or infer answers when possible
                - Escalate or summarize issues when necessary
                - Learn from interactions by updating long-term memory

            Your agent should not only automate, it should decide how to automate!"

            The first UDA-Hub customer Cultpass has the following sample articles to be used as knowledge to be used by the system:
            {samples}

            Provide 10 more example like above?
            """

sample_articles_prompt = PromptTemplate(
    template = sample_articles_template,
    input_variables = ["samples"],
    ).invoke((cultpass_articles)).to_messages()

new_cultpass_articles = llm.with_structured_output(CultpassArticlesListSchema).invoke(sample_articles_prompt).model_dump()['samples']
cultpass_articles = cultpass_articles + new_cultpass_articles

In [11]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

In [12]:
if len(cultpass_articles) < 14:
    raise AssertionError("You should load the articles with at least 14 records")

In [13]:
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb) 
    

**Ticket**

In [14]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [15]:
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

In [16]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()

    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )

    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=ticket_info["role"],
        content=ticket_info["content"],
    )

    session.add_all([user, ticket, metadata, first_message])


# Tests

In [17]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

<Account(account_id='cultpass', account_name='CultPass Card')>


In [18]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    for article in account.knowledge_articles:
        print(article)

<Knowledge(article_id='68e750c7-908a-4697-b2f8-ac5e6c9af211', title='How to Reserve a Spot for an Event')>
<Knowledge(article_id='df9c745c-189d-4a43-b3ed-4b31ba7b558d', title='What's Included in a CultPass Subscription')>
<Knowledge(article_id='81c17e2b-21b5-4982-b019-23f1023dda35', title='How to Cancel or Pause a Subscription')>
<Knowledge(article_id='5ccdd306-902b-4b10-a93a-7bc92397bd29', title='How to Handle Login Issues?')>
<Knowledge(article_id='4043da29-b6c8-4050-867a-56fc64df65a6', title='How to Update Your Profile Information')>
<Knowledge(article_id='4e7f658d-dff3-4971-bf55-c293afcc0d87', title='How to Redeem a Gift Card')>
<Knowledge(article_id='69e07071-0364-4e0e-aa42-e8f72d6fefc7', title='How to Contact Customer Support')>
<Knowledge(article_id='e31721d5-e388-4cc2-a931-878da039aad7', title='How to Change Your Payment Method')>
<Knowledge(article_id='3edd8886-f2f6-4687-ae53-52671dcca92c', title='How to Find Upcoming Events')>
<Knowledge(article_id='0817485e-e313-4370-9e70-7c

In [19]:
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

<User(user_id='cb864290-08f2-4f71-a8c1-f95b287d7110', user_name='Alice Kingsley', external_user_id='a4ab87')>


In [20]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket:udahub.Ticket = user.tickets[0]
    for message in ticket.messages:
        print(message)

<TicketMessage(message_id='695adea0-583c-42d1-bc2f-bae5cab5aa31', role='user', content='I can't log in to my Cultpass ...')>
